![OMNIS2_Logo_projektu.png](../figures/OMNIS2_Logo_projektu.png)

# Ćwiczenie 16: Listy jednokierunkowe: tworzenie, wypisanie, usunięcie

autor: Maciej Przybylski

**Lista jednokierunkowa** jest dynamiczną strukturą danych pozwalającą na dodawanie i usuwanie pojedynczych elementów w dowolnym momencie, bez konieczności przepisywania wszystkich danych, jak to ma miejsce w przypadku tablic. 

Na potrzeby tego ćwiczenia, przygotowana została biblioteka pozwalająca na rysowanie stanu listy oraz wskaźników użytych do jej tworzenia.

**Uruchom dwie poniższe komórki**, aby załadować bibliotekę i wyświetlić przykładową listę.

In [ ]:
#include "../lib/draw_list.h"
#include <iostream>
using namespace std;

In [ ]:
DRAW_LIST_DEMO();

Na rysunku (powyżej),  wskaźniki rysowane są za pomocą strzałek z nazwą nad strzałką i przechowywanym adresem pamięci pod strzałką. Prostokąty oznaczają obszar pamięci zarezerwowany na pole danych i mają wpisaną aktualne dane przechowywane w elemencie listy. 

Do obsługi listy dynamicznej wystarczy adres pierwszego elementu nazywany zwyczajowo **głową listy**. 
Na rysunku adres głowy listy jest zapamiętany w zmiennej `head`.

Adres każdego następnego elementu listy jest przechowywany w elemencie poprzednim, co na rysunku jest pokazane jako strzałka wychodząca z każdego bloku, podpisana jako `nast`. 

Nawet ostatni element ma taką strzałkę (pole z adresem elementu następnego), jednak adres w niej zapisany jest równy zero (`NULL`). 

## Definiowanie elementów listy jednokierunkowej

**Lista dynamiczna jednokierunkowa wymaga zdefiniowania struktury** (bądź klasy), która oprócz pól z danymi **ma wskaźnik na kolejny element tego samego typu co ta struktura.**

Poniżej zdefiniowana jest struktura o nazwie `Element` z polem danych `dane` typu `char` i wskaźnikiem na element następny `nast` typu `Element*`.

In [ ]:
struct Element{
    char dane;
    Element* nast;
};

## Tworzenie listy - wprowadzenie

Zawsze należy zdefiniować co najmniej jeden wskaźnik na pierwszy element listy. 
Lista może być pusta, wtedy wskaźnik na pierwszy element będzie miał adres `NULL`. 
Dlatego, jeżeli nie wiemy czy lista będzie miała jakieś elementy, przygotowujemy wskaźnik z adresem zerowym.

In [ ]:
Element* glowa = nullptr;

W zrozumieniu list pomoże nam funkcja rysująca listę. 

Co się wyświetli po uruchomieniu kodu poniżej?
* Sama strzałka?
* Strzałka i blok, ale pusty w środku?

In [ ]:
DRAW_LIST(glowa);

Elementy listy powstają pojedynczo poprzez dynamiczną alokację za pomocu instrukcji `new`.

In [ ]:
glowa = new Element;

To jest tylko alokacja pamięci na element, ale jeszcze musimy przypisać jakieś dane do pól.
**A w szczególności ustawić adres następnego elementu na `NULL`.**

In [ ]:
glowa->dane = 'A';
glowa->nast = nullptr;
DRAW_LIST(glowa);

Nowy element możemy wstawić zarówno na początku jak i na końcu listy. 
Najpierw dodamy element na końcu. Jego adres musi się znaleźć w polu `nast` ostatniego elementu w aktualnej liście. 
Dla listy z jednym elementem (jak powyżej) można to zrobić tak:

In [ ]:
glowa->nast = new Element;
glowa->nast->dane = 'B';
glowa->nast->nast = nullptr;
DRAW_LIST(glowa);

Dodanie kolejnego elementu tym sposobem nie jest praktyczne. 
Dlatego dobrze jest przygotować pomocniczy wskaźnik przechowujący adres ostatniego elementu. Nazwiemy go `ogon`.

In [ ]:
Element* ogon = glowa->nast;
DRAW_LIST(glowa, ogon);

Do dodawania nowego elementu dobrze jest przygotować jeszcze jedną zmienną wskaźnikową, co zwiększa czytelność kodu.

In [ ]:
Element* nowy = new Element;
nowy->dane = 'C';
nowy->nast = nullptr;
DRAW_LIST(glowa, ogon, nowy);

Na powyższym rysunku widzimy, że udało się utworzyć nowy element, ale nie jest on dowiązany do listy.
Adres nowego elementu musimy wpisać do pola `nast` ostatniego elementu, do którego możemy się dostać przez `ogon`. 

In [ ]:
ogon->nast = nowy;
DRAW_LIST(glowa, ogon, nowy);

`ogon` nie wskazuje już na ostatni element. To można naprawić:

In [ ]:
ogon = nowy;
DRAW_LIST(glowa, ogon, nowy);

Kod z trzech ostatnich komórek można zapętlić. Pętlę przerwie wpisanie znaku `!`

In [ ]:
char znak;
cin >> znak;
while(znak!='!')
{
    Element* nowy = new Element;
    nowy->dane = znak;
    nowy->nast = nullptr;
    ogon->nast = nowy;
    ogon = nowy;
    DRAW_LIST(glowa, ogon, nowy);
    cin >> znak;
}

### Zadanie 16.1 (do wykonania w notatniku)

Poniższy kod zmodyfikować tak, aby z trzech niepowiązanych elmentów stworzyć listę `A -> B -> C`. 
Nie ma potrzeby dodawania nowych wskaźników. 
> Komórkę można uruchamiać wielokrotnie (Ctrl+Enter).

In [ ]:
Element* w1 = new Element;
w1->dane = 'A';
w1->nast = 0;
Element* w2 = new Element;
w2->dane = 'B';
w2->nast = 0;
Element* w3 = new Element;
w3->dane = 'C';
w3->nast = 0;

// Tutaj należy umieścić kod, który z trzech
// niepowiązanych elmentów stowrzy listę A -> B -> C
// ....

DRAW_LIST(w1,w2,w3);

delete w1; delete w2; delete w3;

## Wypisanie listy

Wypisanie listy oznacza konieczność "przejścia" po wszystkich elementach, co zrealizujemy w pętli. 
Potrzebny jest do tego wskaźnik roboczy `p`, który na wstępie przyjmie adres głowy listy, a następnie będzie przyjmował adresy kolejnych elementów, aż osiągnie wartość `NULL` zapisaną w polu `nast` ostatniego elementu. 

**Przejście roboczym wskaźnikiem `p` na następny element odbywa się za pomocą instrukcji `p = p->nast;`**

Poniższy kod zakłada, że wcześniejsze komórki tego notatnika zostały wykonane i mamy listę znaków od adresie pierwszego elementu w zmiennej `glowa`. 
Po uruchomieniu kodu, program rysuje początkowy stan listy i wskaźnika `p`, a następnie czeka na `ENTER`, żeby zacząć wypisanie z animacją. 
> W tym przykładzie nie chcemy rysowania nowych rysunków po każdym kroku, dlatego użyto funkcji `DRAW_LIST_UPDATED`, która oprócz wskaźników przyjmuje jako parametry czas pauzy po narysowaniu rysunku (w milisekundach) oraz identyfikator rysunku do zaktualizowania (zmienna `img_id`).

**Zwróć uwagę na stan wskaźników `glowa` i `p` przed wejściem do pętli oraz po jej zakończeniu.**

In [ ]:
Element* p = glowa;

string img_id = DRAW_LIST(glowa, p);
cout << "Wcisnij ENTER, zeby uruchomic wypisanie";
cin.get();
cout << "Zawartosc listy: ";

while(p!=nullptr)
{
    cout << p->dane << " " << flush;
    p = p->nast;

    int delay = 1000; // pauza po rysowaniu w [ms]
    DRAW_LIST_UPDATED(delay, img_id, glowa, p); // odświeża rysunek zamiast rysować nowy
}


## Usuwanie listy

Lista jednokierunkowa wymaga dynamicznej alokacji pamięci za pomocą `new`, zatem wymaga również zwolnienia tej pamięcia za pomocą `delete`. 
Tutaj również będzie potrzebny pomocniczy wzkaźnik `p`. 

In [ ]:
string img_id = DRAW_LIST(glowa);

cout << "Wcisnij ENTER, zeby uruchomic usuwanie";
cin.get();

while(glowa!=nullptr)
{
    Element* p = glowa;
    glowa = glowa->nast;
    DRAW_LIST_UPDATED(2000, img_id, glowa, p);
    delete p;
}

DRAW_LIST_UPDATED(0, img_id, glowa);

## Przykład kompletnego programu

**Problem:** Napisać program wczytujący z pliku liczby rzeczywiste do listy jednokierunkowej. Wypisać zawartość listy na ekranie. Pamiętać o usunięciu listy na koniec programu. Program napisać z podziałem na funkcje.

In [ ]:
#include <fstream>
#include <cstdlib>
#include "../lib/draw_list.h"

using namespace std;

In [ ]:
struct Liczby{
    // Nazwy pól struktury mogą być dowolne, ale te nazwy pozwalają na użycie funkcji DRAW_LIST
    double dane; 
    Liczby* nast; 
};

In [ ]:
Liczby* wczytajLiczby(string nazwa_pliku)
{
    Liczby* glowa = nullptr;
    Liczby* ogon = nullptr;

    string img_id = DRAW_LIST(glowa, ogon);

    ifstream plik(nazwa_pliku);

    while(plik.good())
    {
        double x;
        if(plik >> x)
        {
            Liczby* nowy = new Liczby;
            nowy->dane = x;
            nowy->nast = nullptr;

            if(glowa==0) // to samo co glowa==nullptr
                glowa = nowy;
            else
                ogon->nast = nowy;
            
            ogon = nowy;

            int delay = 1000; // pauza po rysowaniu w [ms]
            DRAW_LIST_UPDATED(delay, img_id, glowa, ogon, nowy); // odświeża rysunek zamiast rysować nowy
        }
    }

    return glowa;
}

In [ ]:
void wypisz(Liczby* glowa)
{
    Liczby* w = glowa;
    string img_id = DRAW_LIST(glowa, w);
    
    while(w!=nullptr)
    {
        cout << w->dane << " " << flush;
        w = w->nast;
    
        int delay = 1000; // pauza po rysowaniu w [ms]
        DRAW_LIST_UPDATED(delay, img_id, glowa, w); // odświeża rysunek zamiast rysować nowy
    }
    cout << endl << endl;
}

In [ ]:
void usun(Liczby*& glowa)
{
    string img_id = DRAW_LIST(glowa);
    
    while(glowa!=nullptr)
    {
        Liczby* p = glowa;
        glowa = glowa->nast;
        DRAW_LIST_UPDATED(1000, img_id, glowa, p);
        delete p;
    }
    
    DRAW_LIST_UPDATED(0, img_id, glowa);
}

In [ ]:
int main()
{
    Liczby* glowa = 0; // to samo co glowa=nullptr

    cout << "Wczytywanie danych do listy: " << endl;
    glowa = wczytajLiczby("liczby.txt");

    cout << "Wypisanie liczb z listy: " << endl;
    wypisz(glowa);

    cout << "Usuwanie listy" << endl;
    usun(glowa);

    cout << "Zawartosc zmiennej glowa po usuwaniu: " << glowa << endl;
    
    return 0;
}

**Testujemy program:**

In [ ]:
main();

## Zadania do samodzielnego rozwiązania

### Zadanie 16.2

#### Krok 1

Zdefiniuj strukturę dla listy jednokierunkowej o nazwie `Napisy` pozwalającą przechowywać napisy (zmienne typu `string`). 
Będziemy chcieli użyć funkcji `DRAW_LIST`, dlatego pola muszą sie nazywać `dane` i `nast`. 



#### Krok 2

1. Przygotuje 4 zmienne wskaźnikowe typu `Napisy` o nazwach `p1, p2, p3, p4`.
2. Zarezerwuj pamięć na te elementy.
3. Wczytaj do pola `dane` słowa podane przez użytkownika.
4. Do pól `nast` wpisz adres zerowy.
5. Wyświetl na ekranie stan wskaźników za pomocą funkcji `DRAW_LIST(p1, p2, p3, p4);`

#### Krok 3

1. Powiąż ze sobą elementy w kolejności `p4 -> p3 -> p2 -> p1`.
2. Wyświetl na ekranie stan wskaźników za pomocą funkcji `DRAW_LIST(p1, p2, p3, p4);`

#### Krok 4

Zwolnij pamięć po elementach listy korzystając z pętli `while`.

### Zadanie 16.3
Napisać program wczytujący od użytkownika liczby całkowite do listy jednokierunkowej. Przerwać wczytywanie, gdy użytkownik poda liczbę zero, przy czym zero również ma być dodane do listy. Należy wypisać listę na ekranie i pamiętać o zwolnieniu pamięci po liście. Zastosować podział na funkcje.

### Zadanie 16.4

Napisać program, który tworzy jednokierunkową dynamiczną listę znaków losowanych z przedziału podanego przez użytkownika. Dodawanie elementów do listy kończy się w chwili wylosowania znaku stanowiącego górną granicę przedziału. Ostatni znak również należy dodać do listy. 

_Napisać pełen program z użyciem procedur i funkcji, włącznie z wypisaniem i usuwaniem listy._

![logotypy.png](../figures/OMNIS2_logotypy.png)

Niniejsze materiały są objęte licencją <a href="https://creativecommons.org/licenses/by/4.0/">CC BY 4.0</a><img src="https://mirrors.creativecommons.org/presskit/icons/cc.svg" alt="" style="max-width: 1em;max-height:1em;margin-left: .2em;"><img src="https://mirrors.creativecommons.org/presskit/icons/by.svg" alt="" style="max-width: 1em;max-height:1em;margin-left: .2em;">